<a href="https://colab.research.google.com/github/mohammadAliMkh/Pytorch/blob/main/food_vision_modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

os.mkdir("/content/food_vision_project")

In [2]:
%%writefile food_vision_project/fetch_data.py

import os
from pathlib import Path
import requests
from zipfile import ZipFile

def get_data(zip_file_name , raw_path_url):
  """
  Get zip file from url link

    args:
      zip_file_name: name of the zip file (include .zip)
      raw_path_url: url link that raw zip file (include .zip)
    
    return:
      train_dir: train direcotry path
      test_dir: test directory path
  """

  file_name = zip_file_name
  folder_name = file_name.split(".zip")[0]

  parent_dir = Path.cwd()
  data_dir = parent_dir/folder_name

  if Path.exists(data_dir):
    print(f"{folder_name} Folder Already Exists...")
  else:
    os.makedirs(data_dir)
    print(f"{folder_name} Folder Created...")



  # get pizza_steak_sushi.zip file from mrdbourrke github repository
  r = requests.get(raw_path_url)

  with open(file_name , "wb") as f:
    f.write(r.content)

  data_path = parent_dir/file_name
  print(f"{file_name} downloaded...")



  # loading the temp.zip and creating a zip object
  with ZipFile(data_path , 'r') as zObject:
    
      # Extracting all the members of the zip 
      # into a specific location.
      zObject.extractall(
          path=data_dir)
  print(f"{file_name} extracted at {data_dir}")
  # removing the downloaded.zip file from parent directory
  os.remove(data_path)

  train_dir = data_dir/"train"
  test_dir = data_dir/"test"

  return train_dir , test_dir

Writing food_vision_project/fetch_data.py


In [3]:
from food_vision_project import fetch_data
train_dir , test_dir = fetch_data.get_data("pizza_steak_sushi.zip" , "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")

pizza_steak_sushi Folder Created...
pizza_steak_sushi.zip downloaded...
pizza_steak_sushi.zip extracted at /content/pizza_steak_sushi


In [4]:
%%writefile food_vision_project/data_setup.py
import torch
from torch.utils.data import DataLoader

from torchvision.datasets import ImageFolder
from torchvision import transforms

import os
from pathlib import Path


def create_dataLoader(train_dir:Path,
                      test_dir:Path,
                      batch_size:int,
                      transformer:transforms.Compose):
  '''
    Make test/train data Loder object

    inputs:
      train_dir: train data directory
      test_dir: test data directory
      batch_size: batch size of the dataLoader
      transformer: transforms.Compose object 
    
    return:
      train_dataLoader: torch.utils.data.DataLoader object
      test_dataLoader: torch.utils.data.DataLoader object
      class_names: list of the all class names
  '''

  BATCH_SIZE = batch_size
  TRAIN_DIR = train_dir
  TEST_DIR = test_dir
  NUM_WORKERS = os.cpu_count()

  TRANSFORMER = transformer



  train_dataset = ImageFolder(TRAIN_DIR , transform = TRANSFORMER)
  test_dataset = ImageFolder(TEST_DIR , transform = TRANSFORMER)


  train_dataLoader = DataLoader(train_dataset ,
                                batch_size = BATCH_SIZE,
                                shuffle = True,
                                num_workers = NUM_WORKERS,
                                pin_memory = True)

  test_dataLoader = DataLoader(test_dataset,
                              batch_size = BATCH_SIZE,
                              shuffle = False,
                              num_workers = NUM_WORKERS,
                              pin_memory=True)
  
  class_names = train_dataset.classes
  

  return train_dataLoader , test_dataLoader , class_names


Writing food_vision_project/data_setup.py


In [5]:
from torchvision import transforms

transfomer = transforms.Compose([
    transforms.Resize(size = (64 , 64)),
    transforms.ToTensor()
])

In [41]:
from food_vision_project import data_setup
x , y , z = data_setup.create_dataLoader(train_dir , test_dir , batch_size = 32 , transformer = transfomer)

In [10]:
%%writefile food_vision_project/model.py
import torch

Conv_Kernel_Size = 3
Conv_Stride_Size = 1
Conv_Padding_Size = 0

Max_Kernel_Size = 2
Max_Stride_Size = 1

class TinyVGG(torch.nn.Module):
  ''' 
    Tiny VGG Network created from https://poloclub.github.io/cnn-explainer/
  '''

  def __init__(self , input_size:int , hidden_units:int , output_shape:int):
    super().__init__();
    self.conv_block1 = torch.nn.Sequential(
        torch.nn.Conv2d(in_channels = input_size , out_channels = hidden_units,
                        kernel_size = Conv_Kernel_Size ,stride = Conv_Stride_Size,
                        padding = Conv_Padding_Size),

        torch.nn.ReLU(),

        torch.nn.Conv2d(in_channels = hidden_units , out_channels = hidden_units,
                        kernel_size = Conv_Kernel_Size ,stride = Conv_Stride_Size,
                        padding = Conv_Padding_Size),
        torch.nn.ReLU(),

        torch.nn.MaxPool2d(kernel_size = Max_Kernel_Size ,stride = Max_Stride_Size)
    )

    self.conv_block2 = torch.nn.Sequential(
        torch.nn.Conv2d(in_channels = hidden_units , out_channels = hidden_units,
                        kernel_size = Conv_Kernel_Size , stride = Conv_Stride_Size , padding = Conv_Padding_Size),

        torch.nn.ReLU(),

        torch.nn.Conv2d(in_channels = hidden_units , out_channels = hidden_units,
                        kernel_size = Conv_Kernel_Size , stride = Conv_Stride_Size , padding = Conv_Padding_Size),

        torch.nn.ReLU(),

        torch.nn.MaxPool2d(kernel_size = Max_Kernel_Size , stride  = Max_Stride_Size)
    )

    self.last_layer = torch.nn.Sequential(
        torch.nn.Flatten(),

        torch.nn.Linear(in_features = hidden_units * 54 * 54, out_features=output_shape)
    )


  def forward(self , x):
    x = self.conv_block1(x)
    #print(x.shape)
    x = self.conv_block2(x)
    #print(x.shape)
    x = self.last_layer(x)
    #print(x.shape)
    return x
    

Overwriting food_vision_project/model.py


In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [23]:
from food_vision_project import model

cnn = model.TinyVGG(input_size = 3 , hidden_units = 10 , output_shape = 3).to(device)

In [24]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters() , lr = 0.001)

In [45]:
%%writefile food_vision_project/engine.py

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

def train_model(model:torch.nn.Module,
                data:torch.utils.data.DataLoader,
                loss_fn:torch.nn.Module,
                optimizer:torch.optim.Optimizer,
                device = device):
  '''
    Train Model on Each Epoch and Return Train Accuracy and Loss

    args: 
        model: torch.nn.Module
        data: torch.utils.data.DataLoader (train data)
        loss_fn: torch.nn.Module
        optimizer: torch.optim
        device: device agnostic parameter (cpu or cuda)

    outputs:
        acc: List of accuracies in each batch trained
        loss: List of losses in each batch traind
  '''
  
  accuracy = []
  loss = []

  model.train()

  for batch , (X , y) in enumerate(data):

    X , y = X.to(device) , y.to(device)

    train_logits = model(X)

    train_loss = loss_fn(train_logits , y)
    loss.append(train_loss.item())

    optimizer.zero_grad()

    train_loss.backward()

    optimizer.step()

    train_predicts = torch.argmax(torch.softmax(train_logits , dim = 1) , dim = 1)
    train_accuracy = sum(train_predicts == y for train_predicts , y in zip(train_predicts , y))/len(y)
    accuracy.append(train_accuracy)
  
  acc = sum(accuracy)/len(data)
  loss = sum(loss)/len(data)

  return acc.item()*100 , loss


def test_model(model:torch.nn.Module,
               data:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               device = device):
  '''
    Test Model on Each Batch and Return Test Accuracy/Loss

    args:
        model: torch.nn.Module
        data: torch.utils.data.DataLoader (test data)
        loss_fn: torch.nn.Module
        device: device agnostic parameter (cpu or cuda)

    outputs:
        acc: List of test accuracies in each test batch
        loss: List of test losses in each test batch
  '''
  
  accuracy = []
  loss = []

  model.eval()

  with torch.inference_mode():
    for batch , (X , y) in enumerate(data):
      X , y = X.to(device) , y.to(device)

      test_logits = model(X)

      test_loss = loss_fn(test_logits , y)
      loss.append(test_loss)

      test_predicts = torch.argmax(torch.softmax(test_logits , dim = 1) , dim = 1)
      test_accuracy = sum(test_predicts == y for test_predicts , y in zip(test_predicts , y)) / len(y)
      accuracy.append(test_accuracy)
    
    acc = sum(accuracy) / len(data)
    loss = sum(loss) / len(data)

    return acc.item() * 100 , loss.item()

Overwriting food_vision_project/engine.py
